# Processing of Condition Reports (CRs): V1

In [ ]:
import os, sys, time
import re
import pandas as pd
import numpy as np
import spacy
import warnings
warnings.filterwarnings("ignore")
from matplotlib.ticker import MaxNLocator

# Load language model
nlp = spacy.load("en_core_web_lg", exclude=[])

cwd = os.getcwd()
pathToDACKAR   = os.path.join(cwd, '..', '..', 'src')
sys.path.append(pathToDACKAR)

from dackar.text_processing.Preprocessing import Preprocessing
from dackar.utils.utils import getOnlyWords, getShortAcronym
from dackar.text_processing import Abbreviation 
from dackar.utils.nlp.nlp_utils import resetPipeline
from dackar.text_processing.AbbrExpander import AbbrExpander

from dackar.utils.nlp.nlp_utils import generatePatternList
from dackar.pipelines.GeneralEntity import GeneralEntity

## Import Data

In [ ]:
data = pd.read_csv('raw_data/textual/V1_CR.csv')
data['description'] = data['description'].apply(lambda x: x.lower())
data

# Identify equipment IDs

In [ ]:
idLabel = "equip_ID"
ID      = "equip_ID" 

ID_list = pd.read_csv('processed_data/customMBSEobject_ID.csv')['ID'].to_list()
patterns_IDs = generatePatternList(ID_list, label=idLabel, id=ID, nlp=nlp, attr="LEMMA")

pipelines = []
resetPipeline(nlp, pipelines)
nlp.disable_pipes("ner")

ID_ents = GeneralEntity(nlp, patterns_IDs)

ents  = []
for index, act in data['description'].to_frame().iterrows():
    doc = nlp(data['description'].iloc[index].lower())
    newDoc = ID_ents(doc)
    
    newTuple = ()
    for ent in newDoc.ents:
        newTuple = newTuple + (ent.lemma_,)

    ents.append(newTuple)  
data['identifiedID'] = ents
data


# Identify nuclear related entities

In [ ]:
from dackar.utils import tagKeywordListReader as tklr

tagDict = tklr.entityLibrary('../../../DACKAR/data/tag_keywords_lists.xlsx')
tagDict.checker()

nuc_ent_dict = tagDict.getLibrary()

patterns_ents =[]

for key in nuc_ent_dict.keys():
    entLabel = str(key)
    entId    = str(key) 
    patterns_ents.extend(generatePatternList(nuc_ent_dict[key], label=entLabel, id=entId, nlp=nlp, attr="LEMMA"))

In [ ]:
pipelines = []
resetPipeline(nlp, pipelines)
nlp.disable_pipes("ner")

list_entities = []

# General entity object
generalEntity_ents = GeneralEntity(nlp, patterns_ents)

ents  = []
for index, act in data['description'].to_frame().iterrows():
    doc = nlp(data['description'].iloc[index].lower())
    newDoc = generalEntity_ents(doc)
    
    newTuple = ()
    for ent in newDoc.ents:
        newTuple = newTuple + (ent.lemma_,)

    ents.append(newTuple)  
data['NER entities'] = ents
data

# Print on files: Nodes and Edges

In [ ]:
# CR nodes
data[['date', 'ID']].to_csv('processed_data/CR_V1_nodes.csv', index=False)

# NER entity nodes
selected = data['NER entities'].tolist()
selected = list(set(selected))
temp_dict = {'entities':[i for sub in selected for i in sub]}
pd.DataFrame(temp_dict).to_csv('processed_data/entities_V1_nodes.csv', index=False)

# Edges

ent_edges_orig = []
ent_edges_dest = []

ID_edges_orig = []
ID_edges_dest = []

for index, row in data.iterrows():
    if row['NER entities']:
        for ent in row['NER entities']:
            ent_edges_orig.append(row['ID'])
            ent_edges_dest.append(ent)

    if row['identifiedID']:
        for id in row['identifiedID']:
            ID_edges_orig.append(row['ID'])
            ID_edges_dest.append(id)

edges_ent_dict = {'orig':ent_edges_orig,   'dest':ent_edges_dest}
edges_id_dict  = {'orig':ID_edges_orig,    'dest':ID_edges_dest}

pd.DataFrame(edges_ent_dict).to_csv('processed_data/V1_edges_ent.csv', index=False)
pd.DataFrame(edges_id_dict).to_csv('processed_data/V1_edges_id.csv', index=False)
